In [25]:
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
from shapely.geometry import Polygon, LineString, MultiPolygon
from shapely.ops import split
import copy
import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import copy
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier

In [2]:
burned = gpd.read_file("../data/burned/sampleData.csv")
unburned = gpd.read_file("../data/unburned/sampleData.csv")

In [3]:
temp1 = gpd.GeoDataFrame()
temp2 = gpd.GeoDataFrame()
for i in np.unique(burned['FIRE_NAME']):
    temp1 = pd.concat([temp1,burned[burned['FIRE_NAME']==i]])
    temp2 = pd.concat([temp2,unburned[unburned['FIRE_NAME']==i]])

In [4]:
df = pd.concat([temp1,temp2])

In [5]:
classes = {1: "other",       # open water / perennial ice+snow
           2: "developed",
           3: "other",       # barren land
           4: "forest",
           5: "shrub",
           7: "grassland/herbaceous",
           8: "agriculture",
           9: "other"}       # wetland
 
df['landcover'] = df["landcover"].apply(lambda x: classes[int(x)//10])

In [27]:
clean_df = copy.deepcopy(df[['burnSeverity','SR_B3','SR_B6','NDVI','elevation','percent_tree_cover','x_coord','y_coord']])
for i in clean_df.columns:
    clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')
clean_df

,burnSeverity,SR_B3,SR_B6,NDVI,elevation,percent_tree_cover,x_coord,y_coord
0,2,9461.0,15058,0.212274,168,41,-185709.062502,35992.187398
1,1,9716.0,15731,0.270679,138,13,-185825.937502,35843.437398
2,1,9104.0,13635,0.244286,129,29,-185825.937502,37681.562398
3,1,8645.0,11935,0.239246,102,32,-185857.812502,35630.937398
4,1,9706.0,14894,0.229572,165,0,-185868.437502,36310.937398
...,...,...,...,...,...,...,...,...
165052,1,8334.0,9964,0.254991,1624,68,-61923.125001,230745.624897
165053,1,8884.0,11546,0.307321,1726,67,-61923.125001,231199.374897
165054,1,10292.0,9097,0.245669,2078,60,-61923.125001,235131.874897
165055,0,29709.0,9832,-0.040084,2296,0,-61923.125001,236190.624897


In [26]:
for i in clean_df.columns:
    clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
X_train = copy.deepcopy(clean_df.drop(['burnSeverity'],axis=1))
scaler = preprocessing.StandardScaler().fit(X_train.values)
X_scaled = scaler.transform(X_train)
X_scaled

array([[ 4.09555663e-01,  9.80646988e-01,  5.04573752e-01, ...,
         1.11117156e-01, -8.72913561e-02, -1.47025787e+00],
       [ 6.27677398e-01,  1.22420023e+00,  1.12733938e+00, ...,
        -8.22921042e-01, -8.86269727e-02, -1.47134951e+00],
       [ 1.04185235e-01,  4.65674827e-01,  8.45917045e-01, ...,
        -2.89184929e-01, -8.86269727e-02, -1.45785993e+00],
       ...,
       [ 1.12037590e+00, -1.17659057e+00,  8.60658554e-01, ...,
         7.44928790e-01,  1.32730160e+00, -8.81657583e-03],
       [ 1.77292768e+01, -9.10600035e-01, -2.18633244e+00, ...,
        -1.25658163e+00,  1.32730160e+00, -1.04664816e-03],
       [-6.94739079e-01, -1.00469193e+00,  1.35087582e+00, ...,
         1.07851386e+00,  1.32588742e+00, -1.04174779e-01]])

In [31]:
clean_df[['burnSeverity']].values

array([[2],
       [1],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [33]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
pipe = LogisticRegression()
pipe.fit(x_train, y_train.ravel())
print("Logistic Regression: "+str(pipe.score(x_test, y_test)))
importance = pipe.coef_[0]
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

Logistic Regression: 0.7426082246196903
SR_B3: 3.1166127397356993
SR_B6: -3.543937246759957
NDVI: 4.395433400026437
elevation: -0.24727851934017447
percent_tree_cover: -2.6765705602280496
x_coord: 0.8333663314580231
y_coord: 0.3277874332286808


/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['SR_B3', 'NDVI', 'x_coord', 'y_coord']

In [39]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values.ravel())
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
clf = MLPClassifier(max_iter=300).fit(x_train, y_train)
clf.score(x_test, y_test.ravel())

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [22]:
%%time
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
clf = ExtraTreesClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


CPU times: user 17 s, sys: 1.21 s, total: 18.2 s
Wall time: 18.7 s


0.852615898523902

In [23]:
y_pred = clf.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[  482,   592,     3,     0,     0],
       [   91, 38508,  2166,    76,     0],
       [    5,  3476, 11003,  1148,     3],
       [    1,   113,  1535,  6397,   121],
       [    0,     4,     1,   460,   274]])

In [18]:
%%time
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
clf = KNeighborsClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


CPU times: user 2.88 s, sys: 25.5 ms, total: 2.91 s
Wall time: 2.91 s


0.8192569855098633

In [17]:
%%time
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


CPU times: user 47.1 s, sys: 266 ms, total: 47.4 s
Wall time: 47.4 s


0.8532779608480416

In [50]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
scores = []
for i in range(0,10):
    x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    scores.append(clf.score(x_test, y_test))

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0
1
2
3
4
5
6
7
8
9


In [51]:
scores

[0.8521795392648099,
 0.8549481635293941,
 0.8555650852405242,
 0.852871695330956,
 0.8521644923938068,
 0.8534434764290766,
 0.8548277885613687,
 0.8553393821754766,
 0.8536691794941242,
 0.8543914293022766]

In [14]:
y_pred = clf.predict(x_test)

In [15]:
confusion_matrix(y_test, y_pred)

array([[  457,   536,     3,     2,     0],
       [   85, 38826,  2155,    62,     0],
       [    7,  3204, 11136,  1077,     1],
       [    0,    82,  1621,  6340,   103],
       [    0,     2,     8,   469,   283]])

In [ ]:
importance = clf.feature_importances_
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

In [ ]:
rfc = RandomForestClassifier() 

param_grid = { 
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(x_train, y_train)
print(CV_rfc.best_params_)

In [38]:
df['FIRE_NAME'].value_counts()

CARR              35936
FRYING PAN        32879
KING              31144
CAMP              26301
KINCADE           22743
OAK               22166
CALDWELL          20784
HAPPY             18920
BUTTE             18766
ATLAS             18236
ROCKY             16145
BALD              15747
REDWOOD VALLEY    14621
WALKER            14123
COVE              11634
CASCADE            7014
BUCK               5135
Name: FIRE_NAME, dtype: int64

In [39]:
temp = df[df['FIRE_NAME']=='CARR']
clean_df = temp[['burnSeverity','SR_B3','SR_B6','NDVI','elevation','percent_tree_cover','x_coord','y_coord']]
for i in clean_df.columns:
    clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')
for i in clean_df.columns:
    clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
X_train = clean_df.drop(['burnSeverity'],axis=1)
scaler = preprocessing.StandardScaler().fit(X_train.values)
X_scaled = scaler.transform(X_train)
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train = clean_df[['SR_B3','SR_B6','NDVI','elevation','percent_tree_cover','x_coord','y_coord']]
y_train = clean_df[['burnSeverity']]
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
temp = df[df['FIRE_NAME']=='FRYING PAN']
clean_df = temp[['burnSeverity','SR_B3','SR_B6','NDVI','elevation','percent_tree_cover','x_coord','y_coord']]
for i in clean_df.columns:
    clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')
for i in clean_df.columns:
    clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
x_test = clean_df[['SR_B3','SR_B6','NDVI','elevation','percent_tree_cover','x_coord','y_coord']]
y_test = clean_df[['burnSeverity']]
y_pred = clf.predict(x_test)
clf.score(x_test, y_test)

<ipython-input-39-fa21e3a259a8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[i]=pd.to_numeric(clean_df[i], errors='coerce')
/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipyt

0.6053712095866662

In [40]:
confusion_matrix(y_test, y_pred)

array([[    0,   117,     0,     0,     0],
       [    0, 18845,   196,     0,     0],
       [    0,  8327,  1030,     4,     0],
       [    0,  3570,   226,    29,     0],
       [    0,   535,     0,     0,     0]])

In [41]:
((y_pred - y_test['burnSeverity'])**2).mean()

0.8505428997232276